In [514]:
import numpy as np
import tensorflow as tf

In [515]:
learning_rate = 0.00001 
num_epochs = 1000
progress_step = 100

In [516]:
def _parse_line(line):
    defaults = [[0.0], [0.0]]
    fields = tf.decode_csv(line, defaults)
    features = fields[0]
    labels = fields[1]
    features = tf.reshape(features, [1, 1])
    return (features, labels)

In [517]:
dstrain = tf.data.TextLineDataset(['/home/harini/tensorflow/notebooks/immdata-train.csv'])
dstrain = dstrain.map(_parse_line)
print dstrain

dstest = tf.data.TextLineDataset(['/home/harini/tensorflow/notebooks/immdata-test.csv'])
dstest = dstest.map(_parse_line)

iter = tf.data.Iterator.from_structure(dstrain.output_types, dstrain.output_shapes)
(X, Y) = iter.get_next()

training_init_op = iter.make_initializer(dstrain)
test_init_op = iter.make_initializer(dstest)

<MapDataset shapes: ((1, 1), ()), types: (tf.float32, tf.float32)>


In [518]:
# create variables for weight and bias - these are trainable by default and change during the training.
# The weight has 1 feature and 1 output
W = tf.Variable(tf.zeros([1, 1]), name="weight")
b = tf.Variable(tf.zeros([1]), name="bias")

In [519]:
# the predicted value is W.x + b
# the order for matmul is X. W since X has shape (?, 1) and W has shape (1, 1)
pred = tf.matmul(X, W) + b

# estimate the cost - mean-squared error
cost = tf.reduce_sum(tf.pow(pred-Y, 2))

# use a Gradient descent optimizer that minimizes the cost.
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [520]:
# define the init operation
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
   
    try:
        for epoch in range(num_epochs): 
            sess.run(training_init_op)
            _, c, w, bv = sess.run([optimizer, cost, W, b])
            
            if epoch%progress_step == 0:
                print ('epoch: %d cost: %f'% (epoch,c))
                print w, bv
            
        sess.run(test_init_op)
        _, tc, tw, tbv = sess.run([optimizer, cost, W, b])
        print 'test cost: %f' % (tc)
    except tf.errors.OutOfRangeError:
        print 'End of dataset'
        

epoch: 0 cost: 94.672890
[[0.01068354]] [0.0001946]
epoch: 100 cost: 0.000375
[[0.17684117]] [0.00322115]
epoch: 200 cost: 0.000000
[[0.17717186]] [0.00322717]
epoch: 300 cost: 0.000000
[[0.17717241]] [0.0032272]
epoch: 400 cost: 0.000000
[[0.17717241]] [0.00322722]
epoch: 500 cost: 0.000000
[[0.17717241]] [0.00322725]
epoch: 600 cost: 0.000000
[[0.17717241]] [0.00322727]
epoch: 700 cost: 0.000000
[[0.17717241]] [0.00322729]
epoch: 800 cost: 0.000000
[[0.17717241]] [0.00322732]
epoch: 900 cost: 0.000000
[[0.17717241]] [0.00322734]
test cost: 24.969324
